In [ ]:
%pip install --upgrade transformers accelerate peft ai2-olmo bitsandbytes
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
  Using cached accelerate-0.29.1-py3-none-any.whl (297 kB)
  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached ai2_olmo-0.2.5-py3-none-any.whl (118 kB)
  Using cached bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl (102.2 MB)
  Using cached safetensors-0.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached cached_path-1.6.2-py3-none-any.whl (35 kB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached huggingface_hub-0.21.4-py3-none-any.whl (346 kB)
  Using cached pygments-2.17.2-py3-none-any.whl (1.2 MB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manyl

In [ ]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    r=16,
    target_modules=["att_proj", "ff_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=32,
    lora_dropout=0.05
)

2024-04-09 01:36:44.028598: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 01:36:44.028662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 01:36:44.028696: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 01:36:44.036169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Unexpected internal error when monkey patching `Tr

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B")

model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B",
                                             trust_remote_code=True,
                                             cache_dir = "/Volumes/daniel_liden/datasets/h2o_rag",
                                             device_map="auto",
                                             load_in_8bit=True)

The repository for allenai/OLMo-1B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/allenai/OLMo-1B.
You can avoid th... 

In [ ]:
model.model

In [ ]:
from peft import get_peft_model

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

In [ ]:
model.add_adapter(lora_config)

In [ ]:
 Load a subset of the WikiText-2 dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1000]")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])


In [ ]:
# Define the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="/Volumes/daniel_liden/datasets/h2o_rag/output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)